### TODO:
1. For each month, sort articles by viewcount
2. Limit to top 1000 articles

In [1]:
# import libraries
import pandas as pd
import os
from tempfile import mkstemp
from shutil import move
from os import fdopen, remove
import pickle
import numpy as np
import sqlalchemy as sql
import csv
from tqdm import tqdm_notebook
import wikipedia
from multiprocessing import Pool
import datetime


In [ ]:
DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/2016-2018"
lst_data= os.listdir(DIR)


In [ ]:
def replace(file_path):

    fh, abs_path = mkstemp()
    with fdopen(fh, 'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                head = line.split(';')[0:-2]
                tail= line.split(';')[-2:]
                tail = ' '.join(tail)
                head = ';'.join(head)
                final = head + ' ' + tail
                new_file.write(final)
    old = len(open(file_path).readlines())
    new = len(open(abs_path).readlines())
    print("Number of lines: ")
    print("{}, Old: {}, New: {}".format(file_path,old, new))
    
    if old == new:
        move(abs_path, file_path)
        
for file in lst_data:
    if 'pagecounts' in file:
        if 'spaces' not in file:
            replace(os.path.join(DIR, file))

In [ ]:
def create_df(folder_of_files):
    # df of article name, monthly viewcount, day
    
    lst_data = os.listdir(folder_of_files)
    # lst_data=['pagecounts-2016-10-views-ge-5_cleaned']
    columns = ['article_name', 
               'monthly_viewcount', 
               'day']
    df = pd.DataFrame([])
    year_month = []
    for file in lst_data:
        if 'pagecounts' in file:
            next_df = pd.read_csv(os.path.join(folder_of_files, file), sep=' ',dtype={columns[0]:str, columns[1]:int, columns[2]:int}, header=None, names=columns)
            lst_filename = file.split('-')
            yr = lst_filename[1]
            mth = lst_filename[2]
            next_df['year'] = yr
            next_df['month'] = mth
            year_month.append((int(yr), int(mth)))
            next_df.sort_values(by='monthly_viewcount', ascending=False, inplace=True)
            print("Before filtering {}-{}: {}".format(yr,mth,next_df.year.size))
            mask = next_df.article_name.str.contains('Special:') | \
                    next_df.article_name.str.contains('Main_Page') | \
                    next_df.article_name.str.contains('Portal:') | \
                    next_df.article_name.str.contains('Wikipedia:') |\
                    next_df.article_name.str.contains('List of ') |\
                    next_df.article_name.str.contains('Special%:') |\
                    next_df.article_name.str.contains('User:') |\
                    next_df.article_name.str.contains('Help:') |\
                    next_df.article_name.str.contains('Category:') |\
                    next_df.article_name.str.contains('-') |\
                    next_df.article_name.str.contains('404.php') |\
                    next_df.article_name.str.contains('File:')
            
            next_df=next_df[~mask]
            print("After filtering: {}".format(next_df.year.size))
            df = pd.concat([df, next_df.head(1000)], ignore_index = True)
    
    df = df.astype({'article_name': 'str', 'monthly_viewcount': 'int64', 'day': 'int64', 'year': 'int64', 'month':'int64'})
    df['peak_date'] = pd.to_datetime(df[['year', 'month', 'day']])
    return df



In [ ]:

# df = df.astype({'article_name': 'str', 'monthly_viewcount': 'int64', 'day': 'int64', 'year': 'int64', 'month':'int64'})
# df['peak_date'] = pd.to_datetime(df[['year', 'month', 'day']])
# df.head()

df = create_df(os.path.join(DIR, 'final'))

In [ ]:
df.groupby(['year', 'month']).count()

In [2]:
wikipedia.set_lang('en')
wikipedia.set_rate_limiting(rate_limit = True)
def get_article_data(x):
    row = x
    try:
        name = row['article_name']
        page = wikipedia.WikipediaPage(title=name)
        return name, page, row
        
    except:
        name = row['article_name']
        print(name)
        return name, None, row

def insert(lst, con, articles, article_name, year, month, day, view_count, peak_date, summary, page_id):
    lst.append({'title': article_name,
                   'year': year,
                   'month':month,
                   'day':day,
                   'view_count':view_count,
                   'peak_date': peak_date,
                   'summary': summary,
                   'page_id': page_id})
    if len(lst) > 50:
        print("Inserting...")
        con.execute(articles.insert(), lst)
#         except:
#             print("Did not insert")
        lst = []
    return lst
    
    
    

                    

In [ ]:
fromdb_df = pd.read_csv('./2016-2018/top_1000_2016-2018/fromdb_data.csv', names = ['article_name', 'summary', 'page_id'])
fromdb_df.head()

In [ ]:
'404.php' in fromdb_df.article_name
fromdb_df.loc[1].article_name

article_names = set(fromdb_df.article_name.values)

In [ ]:
'404.php' in article_names
# summary = fromdb_df[fromdb_df['article_name']=='adsf'].summary.values[0]
print(fromdb_df[fromdb_df['article_name']=='404.php'].summary.values[0])

In [3]:
def get_final_data(filepath):    
    # df has title, year, month, day, viewcount, peak date
    fromdb_df = pd.read_csv('./2016-2018/top_1000_2016-2018/fromdb_data.csv', names = ['article_name', 'summary', 'page_id'], na_values = '')
    db_dict = {}
    for i, row in fromdb_df.iterrows():
        db_dict[row['article_name']] = (row['summary'], row['page_id'])
        
    article_names = set(fromdb_df.article_name.values)
    new_df = pd.DataFrame(columns = ['title', 'year', 'month', 'day', 'view_count', 'peak_date', 'summary', 'page_id'])
    df = pd.read_csv(filepath)
    exists = []
    for _, row in tqdm_notebook(df.iterrows(), total = len(df)):
        try:
            title = row['article_name']
            year = row['year']
            month = row['month']
            day = row['day']
            view_count =row['monthly_viewcount']
            peak_date = row['peak_date']

            if (title in db_dict):
                summary = db_dict[title][0]
                page_id = db_dict[title][1]
            else:
                _, page, _ = get_article_data(row)
                if page:
                    summary = page.summary
                    page_id = page.pageid
                    
                else:
                    summary = ""
                    page_id = -1
                db_dict[title] = (summary, page_id)
            new_df = new_df.append({'title': title,
                          'year': year,
                          'month': month,
                          'day': day,
                          'view_count': view_count,
                          'peak_date': peak_date,
                          'summary': summary,
                          'page_id': page_id}, ignore_index= True)
        except Exception as e: 
            print("Failed", row['article_name'])
            print(e)


    new_df.to_csv('./2016-2018/top_1000_2016-2018/final_data_0.csv')
    return new_df

In [4]:
get_final_data('./2016-2018/top_1000_2016-2018/partial_view_data0.csv')

Characters_of_Warcraft


/home/justina/Desktop/dv/dv/lib/python3.5/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/justina/Desktop/dv/dv/lib/python3.5/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Alice_Through_the_Looking_Glass_(film)
The_Visit_(2015_film)
New_York
Dualistic
Daredevil_(Marvel_Comics)
Salvatore
Murder_of_JonBen��t_Ramsey
User_talk:199.15.233.174
Calc
2016_Central_Italy_earthquake



,title,year,month,day,view_count,peak_date,summary,page_id
0,UEFA_Euro_2016,2016,6,22,4778955,2016-06-22,"The 2016 UEFA European Championship, commonly ...",7932564
1,Muhammad_Ali,2016,6,4,3227930,2016-06-04,Muhammad Ali (; born Cassius Marcellus Clay Jr...,63747
2,Game_of_Thrones_(season_6),2016,6,27,2128577,2016-06-27,The sixth season of the fantasy drama televisi...,43186937
3,Deaths_in_2016,2016,6,20,1919334,2016-06-20,"This is a list of notable deaths, organized by...",36258669
4,Copa_América_Centenario,2016,6,13,1718182,2016-06-13,The Copa América Centenario (English: Centenni...,34677597
5,Game_of_Thrones,2016,6,27,1619228,2016-06-27,Game of Thrones is an American fantasy drama ...,20715044
6,Christina_Grimmie,2016,6,11,1353079,2016-06-11,"Christina Victoria Grimmie (March 12, 1994 – J...",30963890
7,European_Union,2016,6,24,1292592,2016-06-24,The European Union (EU) is a political and eco...,9317
8,List_of_Game_of_Thrones_episodes,2016,6,27,1197378,2016-06-27,Game of Thrones is an American fantasy drama t...,31120069
9,The_Conjuring_2,2016,6,13,1112128,2016-06-13,The Conjuring 2 (also known as The Conjuring 2...,43360594


In [ ]:
len(set(df.article_name.unique()) - set(fromdb_df.article_name.unique()))

In [ ]:
df.to_csv('./wiki-data/2016-2018/top_1000_2016-2018/view_data.csv')
df_array = np.array_split(df,2)
for i, dframe in enumerate(df_array):
    dframe.to_csv('./wiki-data/2016-2018/top_1000_2016-2018/partial_view_data{}.csv'.format(i))

In [ ]:
x[:2]

## Read the data from top 1000 per month
TOP1000_DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/top_1000_2016-2018"

df1 = pd.DataFrame([])
for f in os.listdir(TOP1000_DIR):
    if 'counts' in f:
        df1 = pd.concat([df1, pd.read_csv(os.path.join(TOP1000_DIR, f))])

In [ ]:
df1.year.size

In [ ]:
def print_rows(limit=10):
    i = 0
    with engine.connect() as con:
        rows = con.execute(articles.select())
        for r in rows:
            print(r.title, int(r.view_count))
            i += 1
            if ( i == limit): return

In [ ]:
print_rows()

In [ ]:
urls_folder_path = "/home/justina/Desktop/dv/data_viz/server/wiki/"

links = open(os.path.join(urls_folder_path, 'urls_all.txt'),'r').readlines()
for i in range(0, len(links),5):
    fh, abs_path = mkstemp()
    with fdopen(fh, 'w') as new_file:
        print(i, i+5)
        for j in links[i:i+5]:
            new_file.write(j)
    move(abs_path, os.path.join(urls_folder_path, 'url{}.txt'.format(i)))

        
    

## Add may 2017 data to db again

In [ ]:
TOP1000_DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/top_1000_per_month"

df1 = pd.DataFrame([])
for f in os.listdir(TOP1000_DIR):
    if 'counts' in f:
        df1 = pd.concat([df1, pd.read_csv(os.path.join(TOP1000_DIR, f))], ignore_index = True)
        
df1.year.size

In [ ]:
df1[(df1.month == 5) & (df1.year == 2017)].head(50)[['article_name', 'peak_date']]

In [ ]:
def add_entries(engine_path, df):
    engine = sql.create_engine(engine_path)
    metadata = sql.MetaData()
    articles = sql.Table('articles', metadata,
        sql.Column('title', sql.String),
        sql.Column('year', sql.Integer),
        sql.Column('month', sql.Integer),
        sql.Column('day', sql.Integer),
        sql.Column('view_count', sql.Integer), 
        sql.Column('peak_date', sql.Date),
        sql.Column('summary', sql.Text),
        sql.Column('page_id', sql.Integer)
    )
    metadata.create_all(engine)

    result = []
    errors = []
    wiki_df = pd.DataFrame(columns=['article_name', 'summary', 'page_id'])

    with engine.connect() as con:
        lst = list(df.iterrows())
    #     with Pool(50) as pool:
    #     for each in tqdm_notebook(pool.imap(get_article_data, lst), total=len(lst)):
        for _, row in lst:
            name = row['article_name']
            year = row['year']
            month = row['month']
            day = row['day']
            view_count = row['monthly_viewcount']
            peak_date = datetime.datetime.strptime(row['peak_date'], '%Y-%m-%d').date()

            if name not in wiki_df.article_name.values:
                name, page, r = get_article_data(row)

                if page:
                    wiki_df = wiki_df.append({'article_name': name, 'summary': page.summary, 'page_id': page.pageid}, ignore_index = True)
                else:
                    wiki_df = wiki_df.append({'article_name': name, 'summary': "", 'page_id': -1}, ignore_index = True)

                result = insert(result,con, articles, name, year, month, day, view_count, peak_date, wiki_df[wiki_df['article_name'] == name].summary.values[0], wiki_df[wiki_df['article_name'] == name].page_id.values[0])
            

        con.execute(articles.insert(), result)
                
        return wiki_df

                

In [ ]:
wiki_df=add_entries('sqlite:///test.db', df1.loc[100:150])

In [ ]:
# Create dataframe of all the data

In [ ]:
wiki_df

In [ ]:
engine = sql.create_engine('sqlite:///test.db')
metadata = sql.MetaData()
articles = sql.Table('articles', metadata,
    sql.Column('title', sql.String),
    sql.Column('year', sql.Integer),
    sql.Column('month', sql.Integer),
    sql.Column('day', sql.Integer),
    sql.Column('view_count', sql.Integer), 
    sql.Column('peak_date', sql.Date),
    sql.Column('summary', sql.Text),
    sql.Column('page_id', sql.Integer)
)
metadata.create_all(engine)

result = []
errors = []
wiki_df = pd.DataFrame(columns=['article_name', 'summary', 'page_id'])

with engine.connect() as con:
    name = df1.loc[0,'article_name']
    year = df1.loc[0,'year']
    month = df1.loc[0,'month']
    day = df1.loc[0,'day']
    view_count = df1.loc[0,'monthly_viewcount']
    peak_date = datetime.datetime.strptime(df1.loc[0,'peak_date'], '%Y-%m-%d').date()

    name, page, row = get_article_data((df1.loc[0]))
    con.execute(articles.insert(), {'title': name,
                                   'year': year,
                                   'month': month,
                                   'day': day,
                                   'view_count': view_count,
                                   'peak_date': peak_date,
                                   'summary': "",
                                   'page_id': -1})

In [ ]:
df1.loc[0,'peak_date']

In [ ]:
wiki_df = add_entries('sqlite:///database_working_add_052017.db',df1)

In [ ]:
df1.dtypes

In [ ]:
wiki_df = add_entries('sqlite:///database_working_add_052017.db',df1[851:])

In [ ]:
df1[df1.article_name == 'Restoration_(Spain)']

In [12]:
columns = ['title', 'year', 'month', 'day', 'view_count', 'peak_date', 'summary', 'page_id']

df_input0 = pd.read_csv('./2016-2018/top_1000_2016-2018/final_data_0.csv', names = columns, skiprows=1)
df_input1 = pd.read_csv('./2016-2018/top_1000_2016-2018/final_data_1.csv', names = columns, skiprows=1)
df_all = pd.concat([df_input0, df_input1], ignore_index=True)

df_all = df_all.reset_index()

print(len(df_all[df_all.page_id == -1]))
print(len(df_all[df_all.summary.isna() & df_all.page_id > 0]))
print(len(df_all))

212
296
35000


In [14]:
mask = (df_all.page_id ==-1)
df_all = df_all[~mask].iloc[:,1:]
print("Remove where page id is -1 ", len(df_all))
df_all = df_all[~df_all.title.str.contains("Template:")]
print("Remove articles with Template: ", len(df_all))

Remove where page id is -1  34788
Remove articles with Template:  34722


In [15]:
df_input = df_all

In [16]:
engine = sql.create_engine('sqlite:///kiru_db.db')
metadata = sql.MetaData()
articles = sql.Table('articles', metadata,
                sql.Column('title', sql.String),
                sql.Column('year', sql.Integer),
                sql.Column('month', sql.Integer),
                sql.Column('day', sql.Integer),
                sql.Column('view_count', sql.Integer),
                sql.Column('peak_date', sql.Date),
                sql.Column('summary', sql.Text),
                sql.Column('page_id', sql.Integer)
                )
metadata.create_all(engine)


with engine.connect() as con:

    for row in tqdm_notebook(df_input.iterrows(), total=len(df_input)):
        row = row[1]
        title = row['title']
        year = row['year']
        month = row['month']
        day = row['day']
        view_count =row['view_count']
        peak_date = row['peak_date']
        summary = row['summary']
        page_id = row['page_id']

        peak_date = datetime.datetime.strptime(peak_date, '%Y-%m-%d').date()
        con.execute(articles.insert(), {'title': title,
                                        'year': year,
                                        'month': month,
                                        'day': day,
                                        'view_count': view_count,
                                        'peak_date': peak_date,
                                        'summary': summary,
                                        'page_id': page_id})